# The spelled-out intro to language modeling: building makemore

[Andrej Karpathy](https://karpathy.ai/)

[YouTube video link](https://youtu.be/PaCmpygFfXo?list=PLAqhIrjkxbuWI23v9cThsA9GvCAUhRvKZ)

> We implement a bigram character-level language model, which we will further complexify in followup videos into a modern Transformer language model, like GPT. In this video, the focus is on (1) introducing torch.Tensor and its subtleties and use in efficiently evaluating neural networks and (2) the overall framework of language modeling that includes model training, sampling, and the evaluation of a loss (e.g. the negative log likelihood for classification).

https://github.com/karpathy/makemore

Requires the training file: `names.txt`

In [ ]:
# Run ONCE to update any new kernel instance.
# You MUST restart the kernel after updating.
!pip install --upgrade pip
!pip install graphviz
!apt-get update
!apt-get install -y graphviz
!pip install torch
print('Complete!')

In [ ]:
words = open('names.txt', 'r').read().splitlines()

In [ ]:
words[:10]

In [ ]:
len(words)

In [ ]:
min(len(w) for w in words)

In [ ]:
max(len(w) for w in words)

In [ ]:
b = {} # store bigram counts in a dictionary
for w in words:
    chs = ['<S>'] + list(w) + ['<E>']
    for ch1,ch2 in zip(chs, chs[1:]): # build a list of bigram pairs (sequential characters)
        bigram = (ch1, ch2)
        b[bigram] = b.get(bigram, 0) + 1
        #print(ch1, ch2)

In [ ]:
sorted(b.items(), key = lambda kv: -kv[1]) # sort (reversed) by the second element in the tuples (i.e. the counts)

## Represent the training set as a 2D array
This will tell us the number of times character X is followed by character Y.

In [ ]:
import torch

In [ ]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
itos

In [ ]:
N = torch.zeros((27, 27), dtype=torch.int32)
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1,ch2 in zip(chs, chs[1:]): # build a list of bigram pairs (sequential characters)
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        N[ix1, ix2] += 1

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(16,16))
plt.imshow(N, cmap='Blues')
for i in range(27):
    for j in range(27):
        chstr = itos[i] + itos[j]
        plt.text(j, i, chstr, ha='center', va='bottom', color='gray')
        plt.text(j, i, N[i,j].item(), ha='center', va='top', color='gray')
plt.title('X is followed by Y N-times')
plt.axis('off')

In [ ]:
N[0] # grab first row

## Sample data set

In [ ]:
# Build probability data set
p = N[0].float()
p = p / p.sum() # normalize to 1.0
p

In [ ]:
g = torch.Generator().manual_seed(2147483647)
ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
itos[ix]

In [ ]:
# We use a generator to make the random distribution repeatable.
g = torch.Generator().manual_seed(2147483647)
p = torch.rand(3, generator=g)
p = p / p.sum()
p

In [ ]:
# multinomial() takes a probability distribution and pulls samples out according to that distribution (with replacement).
torch.multinomial(p, num_samples=100, replacement=True, generator=g)

In [ ]:
# Calculate normalized rows of probabilities
P = (N+1).float() # model smoothing to prevent log(0)=-inf
P /= P.sum(1, keepdim=True) # https://pytorch.org/docs/stable/notes/broadcasting.html

In [ ]:
g = torch.Generator().manual_seed(2147483647)

for i in range(5):
    out = []
    ix = 0
    while True:
        p = P[ix]
        #p = torch.ones(27) / 27.0 # simulate a totally untrained model
        
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[ix])
        if ix == 0:
            break
        
    print(''.join(out))

## Loss function
https://en.wikipedia.org/wiki/Maximum_likelihood_estimation

The log likelihood is commonly used because multiplying the probabilities together gives a very small number. Since its range is [-$\infty$, 0], we want the negative log likelihood so our loss function gets _bigger_ the worse the probabilities are. We also take the average negative log likelihood by dividing by the count of the probabilities.

In [ ]:
# GOAL: maximize likelihood of the data w.r.t. model parameters (statistical modeling)
# equivalent to maximizing the log likelihood (because log is monotonic)
# equivalent to minimizing the negative log likelihood
# equivalent to minimizing the average negative log likelihood

# log(a*b*c) = log(a) + log(b) + log(c)
log_likelihood = 0.0
n = 0
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1,ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        prob = P[ix1, ix2]
        logprob = torch.log(prob) # if the probability is 0, we get -inf which is not good, so we bias all counts to 1 above
        log_likelihood += logprob
        n += 1
        #print(f'{ch1}{ch2}: {prob:.4f} {logprob:.4f}')
        
print(f'{log_likelihood=}')
nll = -log_likelihood
print(f'{nll=}')
print(f'{nll/n}') # loss function (average negative log likelihood)

## Part 2: The Neural Network Approach

I doesn't make sense to feed integers into neurons because (for example) the tanh activation function limits the output to (-1,1). We use one-hot encoding instead.

In [ ]:
# create the training set of bigrams (x,y)... given x, predict y
xs, ys = [], []

for w in words[:1]:
    chs = ['.'] + list(w) + ['.']
    for ch1,ch2 in zip(chs, chs[1:]): # build a list of bigram pairs (sequential characters)
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        print(ch1, ch2)
        xs.append(ix1)
        ys.append(ix2)
        
xs = torch.tensor(xs) # use lower-case 't'ensor so the dtype is inferred correctly
ys = torch.tensor(ys)

In [ ]:
xs

In [ ]:
ys

In [ ]:
# encode the input as one-hot arrays.
import torch.nn.functional as F
xenc = F.one_hot(xs, num_classes=27).float() # one_hot() outputs dtype.int64 so case to float to feed into neural net.
xenc

In [ ]:
plt.imshow(xenc)

In [ ]:
# Create a simple, single layer neural net with 27 inputs
# Neuron: w*x
# We are going to interpret the output as log-counts that we exponentiate to get the character index.

# create a weight tensor by sampling randomly from the normal distribution.
W = torch.randn(27, 27)
xenc @ W  # matrix multiplication. shape: (5, 27) @ (27, 27) = (5, 27). This is the array of the neuron output

In [ ]:
# Decode the outputs
logits = xenc @ W # log-counts
counts = logits.exp() # equivalent to N above
probs = counts / counts.sum(1, keepdims=True) # normalize so the rows sum to 1 so we can compare the output to the training set.
probs

In [ ]:
probs[0].sum()

### SUMMARY

**Softmax**:

Exponentiating logits and then normalizing them is a very common output decoding called softmax. We can interpret the softmax'd output as a probability distribution.

$$
\begin{bmatrix}
1.3 \\
5.1 \\
2.2 \\
0.7 \\
1.1
\end{bmatrix} \Rightarrow \frac{e^{z_i}}{\sum_{j=1}^{K} e^z_j} \Rightarrow
\begin{bmatrix}
0.02 \\
0.90 \\
0.05 \\
0.01 \\
0.02
\end{bmatrix}
$$

In [ ]:
xs

In [ ]:
ys

In [ ]:
# randomly initialize 27 neurons' weights. each neuron receives 27 inputs
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27), generator=g)

In [ ]:
xenc = F.one_hot(xs, num_classes=27).float() # input to the network: one-hot encoding
logits = xenc @ W # predict log-counts
counts = logits.exp() # counts, equivalent to N
probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
# btw: the last 2 lines here are together called the 'softmax'

In [ ]:
probs.shape

In [ ]:
nlls = torch.zeros(5) # size of our input set
for i in range(5):
    # i-th bigram
    x = xs[i].item() # input character index
    y = ys[i].item() # label character index
    print('---------')
    print(f'bigram example {i+1}: {itos[x]}{itos[y]} (indices {x},{y})')
    print('input to neural net:', x)
    print('output probabilities from the neural net:', probs[i])
    print('label (actual next character):', y)
    p = probs[i, y]
    print('probability assigned by the net to the correct character:', p.item())
    logp = torch.log(p)
    print('log likelihood:', logp.item());
    nll = -logp
    print('negative log likelihood:', nll.item())
    nlls[i] = nll
    
print('========')
print('average negative log likelihood, i.e. loss =', nlls.mean().item())

## Optimization + Full Training Loop

In [ ]:
# create the training set of bigrams (x,y)... given x, predict y
xs, ys = [], []

for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1,ch2 in zip(chs, chs[1:]): # build a list of bigram pairs (sequential characters)
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        xs.append(ix1)
        ys.append(ix2)
        
xs = torch.tensor(xs) # use lower-case 't'ensor so the dtype is inferred correctly
ys = torch.tensor(ys)
num = xs.nelement()
print('number of examples: ', num)

g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27), generator=g, requires_grad=True)

In [ ]:
# add regularization loss (aka smoothing). Like adding a spring force pushing W to 0 (see below)
(W**2).mean()

In [ ]:
# gradient decent
for k in range(100):
    # forward pass
    xenc = F.one_hot(xs, num_classes=27).float() # input to the network: one-hot encoding
    logits = xenc @ W
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdims=True)
    
    # loss function (softmax)
    loss = -probs[torch.arange(num), ys].log().mean()
    loss += 0.01 * (W**2).mean() # regularization loss
    print(loss.item())
    
    # backward pass (gradient decent)
    W.grad = None # efficient way to set gradients to 0
    loss.backward()
    
    #update
    W.data += -50 * W.grad # negative sign to reduce the loss; not maximize it

In [ ]:
# finally, sample from the network model
g = torch.Generator().manual_seed(2147483647)

for i in range(5):
    out = []
    ix = 0
    while True:
        # -------
        # BEFORE:
        # p = P[ix]
        # -------
        # NOW:
        xenc = F.one_hot(torch.tensor([ix]), num_classes=27).float()
        logits = xenc @ W # super-simple linear network layer
        counts = logits.exp()
        p = counts / counts.sum(1, keepdims=True)
        # ------
        
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[ix])
        if ix == 0:
            break
        
    print(''.join(out))

In [ ]:
# BEFORE:
junide.
janasah.
p.
cony.
a.